# 11.4 Activity: Producer-Consumer Model Scenario

- Name: Congxin (David) Xu
- Computing ID: cx2rx

### Requirement
Assume you were writing code that simulates a producer-consumer model (e.g., "add" and "remove" methods for a queue—the "add" method adds items to the queue; you cannot add if the queue becomes full; the "remove" method removes items from the queue; you cannot remove if the queue becomes empty). Come up with your own scenario. Write code for the equivalent "add" and "remove" methods in your scenario.

Be sure to:

- Specify clearly what your shared resource (or resources, if applicable) is (e.g., a queue, an integer variable, a string...)
- Use locks [acquire() and release()]
- Use try-finally code structure [remember, where do call the acquire() and release() lock methods?]
- Use condition(s) [using await() and notifyAll()] 
    - Note: You may have more than one condition and each method might have conditions—one method waiting on the other one 
- Comment your code.

### Answer

#### Scenario
This is a fast food take out scenario. Servers at the restaurant prepare the food Customers pick up the food. 

- **The shared resource is the variable `self.backlog`**, which stores the order that is ready to be picked up. Imagine this is the shelf where customer picks up their take out in store. 
- The total number of orders that a backlog can store is 24. 
- Each customer will make at least 1 order and at most 4 orders.
- When the backlog is more than 20 orders, the server will stop preparing the order and wait for the customer to pick up until the total number of orders in backlog is less than 20.
- When the order in backlog is less than the order customer wants to pick up, the customer will stop picking up and waiting for the order to be prepared.

In [2]:
# Import modules
import threading
import time
import numpy

# Build the class
class takeout:
    # Constructor
    def __init__(self):
        self.lock = threading.Lock()
        self.sufficientOrderToPickUp = threading.Condition(self.lock)
        self.sufficientOrderToPrepare = threading.Condition(self.lock)
        self.backlog = 0
    # Method: prepare
    def prepare(self, order):
        
        # Lock the prevent other threads changing the shared resources
        self.lock.acquire()
        
        # Try-Finally block: incase exception and prevent deadlock
        try:
            # While backlog is greater than 20, stop preparing and wait for pickup
            while(self.backlog > 20):
                self.sufficientOrderToPrepare.wait()
            
            # Prepare new orders and add to backlog
            newBacklog = self.backlog + order
            self.backlog = newBacklog
            print("Prepared: %d, new backlog order is %d \n" % (order, newBacklog))
            
            # Notify all threads that orders are ready for pickup
            self.sufficientOrderToPickUp.notifyAll()
        finally:
            # Release the locked object
            self.lock.release()
            
    def pickup(self, order):
       
        # Lock the prevent other threads changing the shared resources
        self.lock.acquire()
        
        # Try-Finally block: incase exception and prevent deadlock
        try:
            # While backlog is less than order, stop picking up and wait for preparing
            while(self.backlog < order):
                self.sufficientOrderToPickUp.wait()
            
            # Pickup new orders and removed from to backlog 
            newBacklog = self.backlog - order
            self.backlog = newBacklog
            print("Picked up: %d, new backlog order is %d \n" % (order, newBacklog))
            
            # Notify all threads that orders can be prepared now
            self.sufficientOrderToPrepare.notifyAll()
        finally:
            # Release the locked object
            self.lock.release()
    
    # Get Backlog orders
    def getBacklog(self):
        return self.backlog

# Initiate prepare
def triggerPrepare(takeout, order, count):
    for i in range(count):
        takeout.prepare(order)
#         time.sleep(1)

# Initiate pick up
def triggerPickup(takeout, order, count):
    for i in range(count):
        takeout.pickup(order)
        time.sleep(1)

if __name__ == '__main__':
    # Build the object and set up parameters
    restaurant_1 = takeout()    
    repetitions = 5
    threads = 12
    
    # Concurrency
    for i in range(threads):
        # Create random order between 1 and 4
        order = numpy.random.randint(1, 5)        
        
        t1 = threading.Thread(target=triggerPrepare, args=(restaurant_1, order, repetitions,))
        t2 = threading.Thread(target=triggerPickup, args=(restaurant_1, order, repetitions,))
        t1.start()
        t2.start()

Prepared: 1, new backlog order is 1 

Prepared: 1, new backlog order is 2 

Prepared: 1, new backlog order is 3 

Prepared: 1, new backlog order is 4 

Prepared: 1, new backlog order is 5 

Picked up: 1, new backlog order is 4 

Prepared: 2, new backlog order is 6 

Prepared: 2, new backlog order is 8 

Prepared: 2, new backlog order is 10 

Prepared: 2, new backlog order is 12 

Prepared: 2, new backlog order is 14 

Picked up: 2, new backlog order is 12 

Prepared: 1, new backlog order is 13 

Prepared: 1, new backlog order is 14 

Prepared: 1, new backlog order is 15 

Prepared: 1, new backlog order is 16 

Prepared: 1, new backlog order is 17 

Picked up: 1, new backlog order is 16 

Prepared: 2, new backlog order is 18 

Prepared: 2, new backlog order is 20 

Prepared: 2, new backlog order is 22 

Picked up: 2, new backlog order is 20 

Prepared: 2, new backlog order is 22 

Picked up: 4, new backlog order is 18 

Prepared: 4, new backlog order is 22 

Picked up: 4, new backlog or